In [1]:
// This top block may not need to be run at all, you can start with the using statements.
// using nightly-build
//#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.Data.Analysis, 0.20.2"
#r "nuget: Microsoft.ML, 4.0.2"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
//#r "nuget:SandDance.InteractiveExtension, 1.0.53"
//#r "nuget: XPlot.Plotly.Interactive, 4.0.6"


Installed Packages Microsoft.Data.Analysis, 0.21.0 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.25323.1 Microsoft.ML, 4.0.2

Loading extensions from `/home/vscode/.nuget/packages/microsoft.data.analysis/0.21.0/interactive-extensions/dotnet/Microsoft.Data.Analysis.Interactive.dll`

Loading extension script from `/home/vscode/.nuget/packages/microsoft.dotnet.interactive.extensionlab/1.0.0-beta.25323.1/interactive-extensions/dotnet/extension.dib`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

ExplainCode 
 This extension generates Sequence diagrams from csharp code using Mermaid kernel.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay an interactive session via automation.

In [2]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive;
//using XPlot.Plotly.Interactive;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using System.Collections.Generic;
using System.IO;
using System.Linq;

In [3]:
var fileExists = File.Exists("draft-info/2025-09-01-top-schools.csv");
if (fileExists)
{
    display("This file exists.  Nothing has blown up.....yet.");
}
var nextDraft = DataFrame.LoadCsv("draft-info/2025-09-01-top-schools.csv");

This file exists.  Nothing has blown up.....yet.

In [4]:
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<int>("SuggestedAuctionValue", nextDraft.Rows.Count()));
nextDraft.Columns.Add(new StringDataFrameColumn("LeagifyPosition", nextDraft.Rows.Count()));

In [5]:
var conferenceToPosition = DataFrame.LoadCsv("conferenceToPosition.csv");


for (int i = 0; i < nextDraft.Rows.Count; i++)
{
    switch((string)nextDraft.Columns["Conference"][i])
    {
        case "SEC":
            nextDraft.Columns["LeagifyPosition"][i] = "SEC";
            break;
        case "ACC":
            nextDraft.Columns["LeagifyPosition"][i] = "ACC";
            break;
        case "Big 12":
            nextDraft.Columns["LeagifyPosition"][i] = "Big 12";
            break;
        case "Pac 12":
            nextDraft.Columns["LeagifyPosition"][i] = "RandomSmallSchool";
            break;
        case "Big Ten":
            nextDraft.Columns["LeagifyPosition"][i] = "Big Ten";
            break;
        default:
            nextDraft.Columns["LeagifyPosition"][i] = "RandomSmallSchool";
            break;
    }
    if ((string)nextDraft.Columns["School"][i] == "Notre Dame")
    {
        nextDraft.Columns["LeagifyPosition"][i] = "Flex";
    }
}


nextDraft



index,School,Conference,ProjectedPoints,NumberOfProspects,SchoolURL,SuggestedAuctionValue,LeagifyPosition
0,Alabama,SEC,204,34,https://d9ioojyvhqpok.cloudfront.net/ipat9gfnm0mjkzxjgjduhgd3dhh5,<null>,SEC
1,Clemson,ACC,202,22,https://d9ioojyvhqpok.cloudfront.net/qdoiydn8n7rrbe24fh91o35g46a2,<null>,ACC
2,Penn State,Big Ten,176,28,https://d9ioojyvhqpok.cloudfront.net/r7ka9jaowmh2i9mbty8xvcw3lszi,<null>,Big Ten
3,Texas,SEC,171,26,https://d9ioojyvhqpok.cloudfront.net/qg9cbgbb8dt3ibmkhhpbnxnmu2ym,<null>,SEC
4,Oregon,Big Ten,165,20,https://d9ioojyvhqpok.cloudfront.net/jtm5dpazszqv0w6emkd6479rdrk8,<null>,Big Ten
5,Ohio State,Big Ten,157,30,https://d9ioojyvhqpok.cloudfront.net/t0cc17koiq4lypyf6v0bjmb48dl1,<null>,Big Ten
6,LSU,SEC,137,28,https://d9ioojyvhqpok.cloudfront.net/kotupd48marnkuttwxr1ueuusuq4,<null>,SEC
7,Georgia,SEC,133,30,https://d9ioojyvhqpok.cloudfront.net/lk6cx1kv1jp68409gl92aapm1amw,<null>,SEC
8,Florida,SEC,114,18,https://d9ioojyvhqpok.cloudfront.net/36khmjuee0xz2lxwa0xvcpewulbr,<null>,SEC
9,Texas A&M,SEC,110,30,https://d9ioojyvhqpok.cloudfront.net/2zs6v615in0lqkjlrpwr0ss7jal3,<null>,SEC


In [6]:
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<double>("ProjectedPointsAboveAverage", nextDraft.Rows.Count()));
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<double>("ProjectedPointsAboveReplacement", nextDraft.Rows.Count()));
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<double>("AveragePointsForPosition", nextDraft.Rows.Count()));
nextDraft.Columns.Add(new PrimitiveDataFrameColumn<double>("ReplacementValueAverageForPosition", nextDraft.Rows.Count()));

nextDraft

index,School,Conference,ProjectedPoints,NumberOfProspects,SchoolURL,SuggestedAuctionValue,LeagifyPosition,ProjectedPointsAboveAverage,ProjectedPointsAboveReplacement,AveragePointsForPosition,ReplacementValueAverageForPosition
0,Alabama,SEC,204,34,https://d9ioojyvhqpok.cloudfront.net/ipat9gfnm0mjkzxjgjduhgd3dhh5,<null>,SEC,<null>,<null>,<null>,<null>
1,Clemson,ACC,202,22,https://d9ioojyvhqpok.cloudfront.net/qdoiydn8n7rrbe24fh91o35g46a2,<null>,ACC,<null>,<null>,<null>,<null>
2,Penn State,Big Ten,176,28,https://d9ioojyvhqpok.cloudfront.net/r7ka9jaowmh2i9mbty8xvcw3lszi,<null>,Big Ten,<null>,<null>,<null>,<null>
3,Texas,SEC,171,26,https://d9ioojyvhqpok.cloudfront.net/qg9cbgbb8dt3ibmkhhpbnxnmu2ym,<null>,SEC,<null>,<null>,<null>,<null>
4,Oregon,Big Ten,165,20,https://d9ioojyvhqpok.cloudfront.net/jtm5dpazszqv0w6emkd6479rdrk8,<null>,Big Ten,<null>,<null>,<null>,<null>
5,Ohio State,Big Ten,157,30,https://d9ioojyvhqpok.cloudfront.net/t0cc17koiq4lypyf6v0bjmb48dl1,<null>,Big Ten,<null>,<null>,<null>,<null>
6,LSU,SEC,137,28,https://d9ioojyvhqpok.cloudfront.net/kotupd48marnkuttwxr1ueuusuq4,<null>,SEC,<null>,<null>,<null>,<null>
7,Georgia,SEC,133,30,https://d9ioojyvhqpok.cloudfront.net/lk6cx1kv1jp68409gl92aapm1amw,<null>,SEC,<null>,<null>,<null>,<null>
8,Florida,SEC,114,18,https://d9ioojyvhqpok.cloudfront.net/36khmjuee0xz2lxwa0xvcpewulbr,<null>,SEC,<null>,<null>,<null>,<null>
9,Texas A&M,SEC,110,30,https://d9ioojyvhqpok.cloudfront.net/2zs6v615in0lqkjlrpwr0ss7jal3,<null>,SEC,<null>,<null>,<null>,<null>


In [7]:
// Let's do some stats on available schools here.
// Get the distinct LeagifyPosition values
string[] leagifyPositions = {"ACC", "SEC", "Big Ten", "Big 12", "RandomSmallSchool", "Flex"};

IDictionary<string, double> draftableAverages = new Dictionary<string, double>();
IDictionary<string, double> replacementAverages = new Dictionary<string, double>();

// Loop over each LeagifyPosition value and get the corresponding rows
foreach (var position in leagifyPositions)
{
    Console.WriteLine("Leagify Position: " + position);
    var rowsForPosition = nextDraft.Rows
        .Where(row => row[6].ToString() == position.ToString());
    
    // Make sure rows are in descending order by potential points.
    var sortedRows = rowsForPosition
    .OrderByDescending(row => row[2])
    .ToList();

    double averageOfTopRows;
    double averageOfRemainingRows;
    if(position.Equals("SEC") || position.Equals("Big Ten"))
    {
        display($"Conference {position} needs a top 12.");
        var topRows = sortedRows.Take(12);
        averageOfTopRows = topRows.Average(row => Convert.ToDouble(row[2]));
        display($"Average points for draftable schools from {position}: {averageOfTopRows}");
        var remainingRows = sortedRows;
        remainingRows.RemoveRange(0,12);
        averageOfRemainingRows = remainingRows.Average(row => Convert.ToDouble(row[2]));
        display($"Average points for replacement value schools from {position}: {averageOfRemainingRows}");


        
    }
    else if(position.Equals("Flex"))
    {
        display("This is Notre Dame, I'm not doing stats on this school.");
        averageOfTopRows = sortedRows.Average(row => Convert.ToDouble(row[2]));
        averageOfRemainingRows = sortedRows.Average(row => Convert.ToDouble(row[2]));
    }
    else
    {
        display($"Conference {position} only needs a top 6");
        var topRows = sortedRows.Take(6);
        averageOfTopRows = topRows.Average(row => Convert.ToDouble(row[2]));
        display($"Average points for draftable schools from {position}: {averageOfTopRows}");

        var remainingRows = sortedRows;
        remainingRows.RemoveRange(0,6);
        averageOfRemainingRows = remainingRows.Average(row => Convert.ToDouble(row[2]));
        display($"Average points for replacement value schools from {position}: {averageOfRemainingRows}");
    }

    draftableAverages.Add(position, Math.Truncate(averageOfTopRows * 100) / 100);
    replacementAverages.Add(position, Math.Truncate(averageOfRemainingRows * 100) / 100);
}





Leagify Position: ACC


Conference ACC only needs a top 6

Average points for draftable schools from ACC: 70.5

Average points for replacement value schools from ACC: 15.545454545454545

Leagify Position: SEC


...

Conference RandomSmallSchool only needs a top 6

Average points for draftable schools from RandomSmallSchool: 10

Average points for replacement value schools from RandomSmallSchool: 2.0714285714285716

Leagify Position: Flex


This is Notre Dame, I'm not doing stats on this school.

In [8]:
display(draftableAverages);
display(replacementAverages);

// Iterate over the rows and set the value of the new column based on the value in the Name column
for (int i = 0; i < nextDraft.Rows.Count; i++)
{
    string leagifyPosition = "";
    switch((string)nextDraft.Columns["LeagifyPosition"][i])
    {
        
        case "SEC":
            leagifyPosition = "SEC";
            break;
        case "ACC":
            leagifyPosition = "ACC";
            break;
        case "Big 12":
            leagifyPosition = "Big 12";
            break;
        case "Big Ten":
            leagifyPosition = "Big Ten";
            break;
        case "RandomSmallSchool":
            leagifyPosition = "RandomSmallSchool";
            break;
        case "Flex":
            leagifyPosition = "Flex";
            break;
        default:
            leagifyPosition = "RandomSmallSchool";
            break;

    }

    nextDraft.Columns["AveragePointsForPosition"][i] = draftableAverages[leagifyPosition];
    var projectedPointsAboveAverage = (Single)nextDraft.Columns["ProjectedPoints"][i] - draftableAverages[leagifyPosition];
    nextDraft.Columns["ProjectedPointsAboveAverage"][i] = Math.Truncate(projectedPointsAboveAverage * 100) / 100;

    nextDraft.Columns["ReplacementValueAverageForPosition"][i] = replacementAverages[leagifyPosition];
    var projectedPointsAboveReplacement = (Single)nextDraft.Columns["ProjectedPoints"][i] - replacementAverages[leagifyPosition];
    nextDraft.Columns["ProjectedPointsAboveReplacement"][i] = Math.Truncate(projectedPointsAboveReplacement * 100) / 100;
    display($"School {(string)nextDraft.Columns["School"][i]} in conference {(string)nextDraft.Columns["Conference"][i]} has Leagify position {leagifyPosition}");
}

display(nextDraft);

key,value
ACC,70.5
SEC,106.33
Big Ten,77.5
Big 12,52.33
RandomSmallSchool,10
Flex,101


key,value
ACC,15.54
SEC,16
Big Ten,9.83
Big 12,11.5
RandomSmallSchool,2.07
Flex,101


School Alabama in conference SEC has Leagify position SEC

School Clemson in conference ACC has Leagify position ACC

School Penn State in conference Big Ten has Leagify position Big Ten

...

School Florida A&M in conference FCS has Leagify position RandomSmallSchool

School Air Force in conference FBS - Mountain West has Leagify position RandomSmallSchool

School Stephen F. Austin in conference FCS has Leagify position RandomSmallSchool

School East Tennessee State in conference FCS has Leagify position RandomSmallSchool

index,School,Conference,ProjectedPoints,NumberOfProspects,SchoolURL,SuggestedAuctionValue,LeagifyPosition,ProjectedPointsAboveAverage,ProjectedPointsAboveReplacement,AveragePointsForPosition,ReplacementValueAverageForPosition
0,Alabama,SEC,204,34,https://d9ioojyvhqpok.cloudfront.net/ipat9gfnm0mjkzxjgjduhgd3dhh5,<null>,SEC,97.67,188,106.33,16
1,Clemson,ACC,202,22,https://d9ioojyvhqpok.cloudfront.net/qdoiydn8n7rrbe24fh91o35g46a2,<null>,ACC,131.5,186.46,70.5,15.54
2,Penn State,Big Ten,176,28,https://d9ioojyvhqpok.cloudfront.net/r7ka9jaowmh2i9mbty8xvcw3lszi,<null>,Big Ten,98.5,166.17,77.5,9.83
3,Texas,SEC,171,26,https://d9ioojyvhqpok.cloudfront.net/qg9cbgbb8dt3ibmkhhpbnxnmu2ym,<null>,SEC,64.67,155,106.33,16
4,Oregon,Big Ten,165,20,https://d9ioojyvhqpok.cloudfront.net/jtm5dpazszqv0w6emkd6479rdrk8,<null>,Big Ten,87.5,155.16,77.5,9.83
5,Ohio State,Big Ten,157,30,https://d9ioojyvhqpok.cloudfront.net/t0cc17koiq4lypyf6v0bjmb48dl1,<null>,Big Ten,79.5,147.16,77.5,9.83
6,LSU,SEC,137,28,https://d9ioojyvhqpok.cloudfront.net/kotupd48marnkuttwxr1ueuusuq4,<null>,SEC,30.67,121,106.33,16
7,Georgia,SEC,133,30,https://d9ioojyvhqpok.cloudfront.net/lk6cx1kv1jp68409gl92aapm1amw,<null>,SEC,26.67,117,106.33,16
8,Florida,SEC,114,18,https://d9ioojyvhqpok.cloudfront.net/36khmjuee0xz2lxwa0xvcpewulbr,<null>,SEC,7.67,98,106.33,16
9,Texas A&M,SEC,110,30,https://d9ioojyvhqpok.cloudfront.net/2zs6v615in0lqkjlrpwr0ss7jal3,<null>,SEC,3.67,94,106.33,16


In [9]:
//nextDraft.Columns.Remove("SchoolURL");
DataFrame.WriteCsv(nextDraft,"auctionDraftAverages-2026-sample.csv",',');